In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
alldata = pd.read_excel("AllData.xlsx")

### Cleaning column by column

#### Column1 - Emotion

In [4]:
cleaned_column = alldata.iloc[:,1].str.upper().str.strip().str.replace('PP','P').str.replace('M','N')
cleaned_column.value_counts()

P          16920
N          13055
O              8
L              7
H              2
A              1
照片无法识别         1
图片无法读取         1
B              1
C              1
无图案            1
什么图都没有         1
什么图案都没有        1
F              1
Name: Emotion, dtype: int64

In [5]:
#Getting just P and N values
cleaned_column[~cleaned_column.isin(['P','N'])] = None
print(cleaned_column.value_counts())
alldata.iloc[:,1] = cleaned_column

P    16920
N    13055
Name: Emotion, dtype: int64


In [6]:
alldata.iloc[:,1].value_counts()

P    16920
N    13055
Name: Emotion, dtype: int64

#### Column2 - Emotional strength

In [7]:
c2_clean = alldata.iloc[:,2].str.strip().str.upper().replace('LL','L').replace('MM','M')
c2_clean[~c2_clean.isin(['L','M','H'])] = None
alldata.iloc[:,2] = c2_clean

#### Column3 - Concrete And Abstract

In [8]:
c3_clean = alldata.iloc[:,3].str.strip().str.upper().replace('AA','A').replace(['M9','M1','MM'],'M').replace('CA','C')
c3_clean[~c3_clean.isin(['C','A','M'])] = None
alldata.iloc[:,3] = c3_clean

#### Columns 4-7,9,10 - Color/Texture

In [9]:
def clean_numeric_cols(alldata_column):
    c3_clean = alldata_column.astype('str').str.strip()
    c3_list = []
    
    for each in c3_clean:

        #Replacing unwanted characters with what is required
        each_string = each.replace('，',',').replace('O','0').replace('.',',')
        each_string = re.sub('，|。',',',each_string)
        
        #U - Undefined - This has been allocated the number 99
        #So 99 means - undefined/not present
        
        #Changed '63' to '6,3' in Major_Background_color color manually by verifying tags of other images
        each_string = re.sub('77','7',each_string)
        each_string = re.sub('99','9',each_string)
        each_string = re.sub('U','99',each_string)
        each_string = re.sub('[A-Z]','',each_string)

        set_values = list(set(each_string.split(',')))

        #Number set to 20 so that we consider all the numbers between 1-20
        set_values = [each for each in set_values if each in list(map(str, range(100 + 1)))]

        c3_list.append(','.join(set_values))
        
    return pd.DataFrame(c3_list).iloc[:,0]    

#change the U Category to Others

In [10]:
alldata.iloc[:,[4,5,6,7,9,10]] = alldata.iloc[:,[4,5,6,7,9,10]].apply(clean_numeric_cols)

#### Column 8 

In [11]:
col8 = alldata.iloc[:,8].astype(str).str.strip().replace(['21','22',',2'],'2').replace('33','3').replace('7，1','1')
col8[~col8.isin(['1','2','3'])] = None
alldata.iloc[:,8] = col8

In [12]:
alldata.iloc[:,8].value_counts()

2    13086
3    10788
1     6077
Name: Design_contrast_color, dtype: int64

#### Column 11 - Objects

In [13]:
alldata.iloc[:,11] = alldata.iloc[:,11].str.replace('[0-9A-Z]','').replace('，',',')

In [14]:
#alldata.head()

### One hot encoding - Creating the response variable for the data required for CNN

##### Two major procedure done in this function

* Aggregating three types of lables to one label
* Creating one hot encoding for each of the lablel

In [15]:
#Input the column name
def final_one_hot(colname):
    
    #Selecting the position of the column and the main Index id
    select_cols = np.where(alldata.columns == colname)[0][0]
    colname_id = alldata.columns[0]
    
    #Selecting only the required column and dropping the rest
    newdata = alldata.iloc[:,[0,select_cols]]
    newdata.index = newdata.iloc[:,0]
    newdata = newdata.drop(colname_id,axis = 1).iloc[:,0]

    #Creating one observation for each of the records
    df_one_hot = newdata.str.split(",", expand = True).stack().str.get_dummies().sum(level=0)
    
    #Making the values as one hot representation since we have values apart from 1 and 0
    df_one_hot_max_value = df_one_hot.apply(lambda x: x == max(x), axis = 1).apply(lambda x: x*1)
    df_one_hot_max_value.columns =  colname+ ' - ' + df_one_hot_max_value.columns
    return(df_one_hot_max_value)

In [16]:
alldata.columns

Index(['Index', 'Emotion', 'Emotional_strength', 'Theme',
       'Background_color_all', 'Major_Background_color', 'Design_color_all',
       'Major_Design_color', 'Design_contrast_color', 'Graphics',
       'Major_Graphics', 'Objects'],
      dtype='object')

In [17]:
one_hot_output = final_one_hot('Major_Background_color')

In [18]:
one_hot_output.to_csv('Major_Background_one_hot.csv')

#### Questions to answer

1) There are columns like 'Major Background color' which should have only one lable for an ID. But, there are around ~500 cases for the major background color where we see each of the label for a particular index is different. So, we end up with ~3 labels for each index for those cases
    * So, do we have all three of them or select one?